# Part 1: Cleaning and Data Wrangling

The goal of this project is predict wine scores from wine reviews, geography, variety, and vintage. I downloaded a wine dataset from [Kaggle](https://www.kaggle.com/zynicide/wine-reviews) which will be used to predict the wine scores. Because I am predicting a target value from multiple independent variables this will be a Supervised Learning model.

In this jupyter notebook I will import the csv datafile and clean the dataset. This will include removing/filling missing values, dropping columns, and conduct feature engineering. I will need to determine which independent variables to keep and which independent variables are not important for predicting the target value of review score. 

I thought this dataset would be interesting to work with because it is as text dataset and can be utilized in many different ways. As well determining how to clean and model data with text is something I wanted to get familiar with.

As a reference this dataset was originally scraped from www.winemag.com 

In [43]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# View 100 rows and columns in the dataframe
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',100)

### A. Preliminary Work: Explore the Data
I will first explore the data to make some quick observations before cleaning the data.

In [3]:
# Import .csv file and save as df dataframe
df = pd.read_csv('data/winemag_data_130k_v2.csv', index_col=[0])

In [4]:
# Quick view of first 5 rows of wine_df
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


#### Define each column: 
- country: Name of the Country where the wine is grown
- description: Description of the wine review
- designation: The vineyard within the winery where the grapes that made the wine
- points: Rating of the wine ranging 80 - 100
- price: Price of a bottle of wine
- province: Name of the Province/State where the wine is grown
- region_1: Region within the province where the wine is grown
- reigon_2: A sub-region within the region where the wine is grown
- taster_name: Name of the person who rated the wine
- taster_twitter_handle: The twitter handle of the person who rated the wine.
- tile: Name of the wine
- variety: The blend/type of wine
- winery: Name of the winery where the wine is made.


#### Things to note: 

- points is our target and will be set as the dependent variable
- taster_twiter_handle will be deleted as it will not be included in the model
- Each row of data is a different review
- Each column describes information about the wine, review, and reviewer
- I can extract the Vintage (year) of the wine from the title column
- NaN values are present
- taster_twiter_handle will be deleted as it will not be included in the model
- region_2 will be removed from the dataset because text data will really enlarge the size of the dataset, as well region_2 seems to sometimes replicate the same value as in region_1 (see row 2 from above). Having this similarity can cause a multi-colinearity problem in the model.
- I have also decided to drop region_1 because the dataset will become very large once I use NLP on the text columns. I can always include region_1 later on to see how the models run.
- I will drop designation for the same reason for dropping region_1

#### Check the data type

In [5]:
# Check data types for each column to make sure data type "makes sense"
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


The data type is correct for all columns, therefore I do not need to make any changes to the data type.

#### Check the shape of the dataset

In [6]:
df.shape

(129971, 13)

Dataset has 129,971 rows and 14 columns. The dataset shape is good for modeling because the columns are much less than the rows which is important for fitting a model. However, the dataset will frow in columns once I transform the data from text data to numerical data, a process that is important for being able to run predictive models on text data.

### B. Cleaning Data

#### Step 1: Make a copy of the dataset

In [7]:
# copy wine dataframe 
df_clean = df.copy()

#### Step 2: Create train and test dataset

The train dataset will be used to train the model and the test dataset will be used to test how will the model runs on a dataset that is not trained on. I split the datatset before even cleaning so that I can make sure all steps can be replicated for a future dataset. As well, I will not be "peaking" into the test dataset to make decisions on how to clean the data and how to model the data.

In [8]:
# Define X and Y
y = df_clean['points']
X = df_clean.drop('points', axis = 1)

In [9]:
# View X dataset
X.head()

,country,description,designation,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [10]:
# View y dataset
y.head()

0    87
1    87
2    87
3    87
4    87
Name: points, dtype: int64

In [11]:
# split data into train and test sets

# Import train_test_split package
from sklearn.model_selection import train_test_split

# Split data into train and test, where text_size is 30 percent, andsp train set is 70%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify = y)


In [12]:
# View size of Train/Test set
print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (90979, 12)
y_train shape (90979,)
X_test shape (38992, 12)
y_test shape (38992,)


#### Step 2: Drop Columns

I dropped all columns that seemed like would be less helpful for the model. This is a qualitative decision. For a more quantitative decision, I could look at the variance for each column. If there is high variance than I might want to consider keeping that column, while a column with low variance, I might want to consider dropping.

In [13]:
# Drop columns
X_train = X_train.drop(['designation', 'taster_twitter_handle', 'winery', 'taster_name', 'region_2', 'region_1'], axis=1)

# Check to make sure columns were dropped
X_train.head()

,country,description,price,province,title,variety
124134,France,An impressively structured blend of Tannat and...,25.0,Southwest France,Producteurs Plaimont 2008 Château de Crouseill...,Tannat-Cabernet Franc
104614,Argentina,"Roasted, earthy aromas of herbs and tomato pla...",14.0,Other,Michel Torino 2013 Coleccion Cabernet Sauvigno...,Cabernet Sauvignon
79944,Portugal,The southern coastal region of Algarve is bett...,35.0,Algarve,Casa Santos Lima 2014 Al-Ria Reserva Red (Alga...,Portuguese Red
7082,Australia,Aromas of sour cherry and cooked strawberries ...,30.0,Victoria,Giant Steps 2006 Sexton Vineyard Pinot Noir (Y...,Pinot Noir
23585,Italy,The Calderara Sottana cru delivers wines with ...,46.0,Sicily & Sardinia,Tenuta delle Terre Nere 2007 Calderara Sottana...,Red Blend


In [14]:
# Reset index
X_train = X_train.reset_index()
X_train.drop('index', axis = 1, inplace = True)

In [15]:
X_train.head()

,country,description,price,province,title,variety
0,France,An impressively structured blend of Tannat and...,25.0,Southwest France,Producteurs Plaimont 2008 Château de Crouseill...,Tannat-Cabernet Franc
1,Argentina,"Roasted, earthy aromas of herbs and tomato pla...",14.0,Other,Michel Torino 2013 Coleccion Cabernet Sauvigno...,Cabernet Sauvignon
2,Portugal,The southern coastal region of Algarve is bett...,35.0,Algarve,Casa Santos Lima 2014 Al-Ria Reserva Red (Alga...,Portuguese Red
3,Australia,Aromas of sour cherry and cooked strawberries ...,30.0,Victoria,Giant Steps 2006 Sexton Vineyard Pinot Noir (Y...,Pinot Noir
4,Italy,The Calderara Sottana cru delivers wines with ...,46.0,Sicily & Sardinia,Tenuta delle Terre Nere 2007 Calderara Sottana...,Red Blend


#### Step 3: Missing Values

In [16]:
# Number of null values for each column
X_train.isna().sum()

country          42
description       0
price          6313
province         42
title             0
variety           0
dtype: int64

In [17]:
# Check to make sure there are no missing values for y_train
y_train.isna().sum()

0

In [18]:
# To determine how to fill the nans, calculate the percent of nans for each column
X_train.isna().sum() / df.shape[0]

country        0.000323
description    0.000000
price          0.048572
province       0.000323
title          0.000000
variety        0.000000
dtype: float64

All columns are missing less than 10% of its data therefore for the text dataset, I will fill the columns be the most frequent value in that column and for numeric data I will fill the column by the median. There is no need to drop columns or remove rows due to missing value. 


#### 3a. Fill Variety Missing Values

In [19]:
# Replace variety nan with most frequent variety
X_train['variety'].fillna(X_train['variety'].describe()['top'], inplace = True)

# Replace variety nan with most frequent variety
X_train['country'].fillna(X_train['country'].describe()['top'], inplace = True)

# Replace variety nan with most frequent variety
X_train['province'].fillna(X_train['province'].describe()['top'], inplace = True)

# Replace variety nan with most frequent variety
X_train['price'].fillna(X_train['price'].median(), inplace = True)

# Check to make sure missing values are filled in variety
X_train.isna().sum()

country        0
description    0
price          0
province       0
title          0
variety        0
dtype: int64

## B. Data Wrangling - Feature Engingineering

Feature engineering is a way to extract information out of the 

### 1. Extract year of the wine

In [20]:
# View the first title to get an idea of the format and how to extract the year 2013.
X_train['title'][0]

'Producteurs Plaimont 2008 Château de Crouseilles Tannat-Cabernet Franc (Madiran)'

In [21]:
# Use regex to extract the year from title

import re
year = []

regex = '([1][9][0-9][0-9]|[2][0-1][0-2][0-9])'             
#
for i in np.arange(0, X_train.shape[0]):
    year.append(re.findall(regex, X_train['title'][i]))
    #print(re.findall(regex, wine_clean['title'][i]), i)
    
vintage = pd.DataFrame(year)

from datetime import datetime as dt

#Change Review_date to a datetime
vintage[0] = pd.to_datetime(vintage[0])
vintage[1] = pd.to_datetime(vintage[1])

The output is two columns because there are some titles that lists two years. One year is the actual year when the wine was made and the other year could be the year the winery started producing wine. Below are the steps I took to remove the second year that is not associated to the year of when the wine was made.

In [22]:
# Check to make sure the columns are changed to datetime
vintage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90979 entries, 0 to 90978
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       87738 non-null  datetime64[ns]
 1   1       55 non-null     datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.4 MB


In [23]:
# The output is two columns 
vintage[0] = pd.DatetimeIndex(vintage[0]).year
vintage[1] = pd.DatetimeIndex(vintage[1]).year

vintage.head()

,0,1
0,2008.0,NaN
1,2013.0,NaN
2,2014.0,NaN
3,2006.0,NaN
4,2007.0,NaN


I fill the Nan with zeros because I will create a threshold to remove the year that is not assiciated to the vintage.

In [24]:
# Replace all NaN with Zero
vintage.fillna(0, inplace = True)

vintage.isna().sum()

0    0
1    0
dtype: int64

In [25]:
# The threshold that I create for finding the correct vintage is assuming that
# if there are two years in the same row, the more recent year is the year that is associated with the vintage
# For example if a row has a year of 2000 and a year 1930, most likely the wine was made in 2000 not in 1930

for i in np.arange(0, vintage.shape[0]):
    if vintage[0].iloc[i] < vintage[1].iloc[i]:
        vintage[0].iloc[i] = vintage[1].iloc[i]
        
    elif vintage[0].iloc[i] == 0:
        vintage[0].iloc[i] = vintage[0].median()
        
# Ceck to make sure all 0 values are removed
for i in np.arange(0, vintage.shape[0]):
    if vintage[0].iloc[i] == 0:
        print(i)
        
X_train['vintage'] = vintage[0]

# Now that the year has been extracted from the wine, drop the title column which will not be used to run a predictive model.
X_train.drop('title', axis = 1, inplace = True)

In [26]:
X_train.head()

,country,description,price,province,variety,vintage
0,France,An impressively structured blend of Tannat and...,25.0,Southwest France,Tannat-Cabernet Franc,2008.0
1,Argentina,"Roasted, earthy aromas of herbs and tomato pla...",14.0,Other,Cabernet Sauvignon,2013.0
2,Portugal,The southern coastal region of Algarve is bett...,35.0,Algarve,Portuguese Red,2014.0
3,Australia,Aromas of sour cherry and cooked strawberries ...,30.0,Victoria,Pinot Noir,2006.0
4,Italy,The Calderara Sottana cru delivers wines with ...,46.0,Sicily & Sardinia,Red Blend,2007.0


In [27]:
# Add the points column back into the dataset to get ready for exporting the file
y_train = y_train.reset_index().drop('index', axis = 1)
X_train['points'] = y_train
X_train.head()

,country,description,price,province,variety,vintage,points
0,France,An impressively structured blend of Tannat and...,25.0,Southwest France,Tannat-Cabernet Franc,2008.0,90
1,Argentina,"Roasted, earthy aromas of herbs and tomato pla...",14.0,Other,Cabernet Sauvignon,2013.0,85
2,Portugal,The southern coastal region of Algarve is bett...,35.0,Algarve,Portuguese Red,2014.0,90
3,Australia,Aromas of sour cherry and cooked strawberries ...,30.0,Victoria,Pinot Noir,2006.0,82
4,Italy,The Calderara Sottana cru delivers wines with ...,46.0,Sicily & Sardinia,Red Blend,2007.0,93


In [28]:
# Export Train dataset as a csv to be imported into another jupyter notebook.
X_train.to_csv (r'train_clean.csv', index = False)

# Repeat for test dataset

In [41]:
# Drop columns
X_test = X_test.drop(['designation', 'taster_twitter_handle', 'winery', 'taster_name', 'region_2', 'region_1'], axis=1)

# Reset index
X_test = X_test.reset_index()
X_test.drop('index', axis = 1, inplace = True)

# Replace variety nan with most frequent variety
X_test['variety'].fillna(X_test['variety'].describe()['top'], inplace = True)

# Replace variety nan with most frequent variety
X_test['country'].fillna(X_test['country'].describe()['top'], inplace = True)

# Replace variety nan with most frequent variety
X_test['province'].fillna(X_test['province'].describe()['top'], inplace = True)

# Replace variety nan with most frequent variety
X_test['price'].fillna(X_test['price'].median(), inplace = True)

import re
year = []

regex = '([1][9][0-9][0-9]|[2][0-1][0-2][0-9])'             
#
for i in np.arange(0, X_test.shape[0]):
    year.append(re.findall(regex, X_test['title'][i]))
    #print(re.findall(regex, wine_clean['title'][i]), i)
    
vintage = pd.DataFrame(year)

from datetime import datetime as dt

#Change Review_date to a datetime
vintage[0] = pd.to_datetime(vintage[0])
vintage[1] = pd.to_datetime(vintage[1])

vintage[0] = pd.DatetimeIndex(vintage[0]).year
vintage[1] = pd.DatetimeIndex(vintage[1]).year

# Replace all NaN with Zero
vintage.fillna(0, inplace = True)

for i in np.arange(0, vintage.shape[0]):
    if vintage[0].iloc[i] < vintage[1].iloc[i]:
        vintage[0].iloc[i] = vintage[1].iloc[i]
        
    elif vintage[0].iloc[i] == 0:
        vintage[0].iloc[i] = vintage[0].median()
        
# Ceck to make sure all 0 values are removed
for i in np.arange(0, vintage.shape[0]):
    if vintage[0].iloc[i] == 0:
        print(i)
        
X_test['vintage'] = vintage[0]

# Now that the year has been extracted from the wine, drop the title column which will not be used to run a predictive model.
X_test.drop('title', axis = 1, inplace = True)

# add y back to column
y_test = y_test.reset_index().drop('index', axis = 1)
X_test['points'] = y_test
X_test.head()

# Export test dataset as a csv to be imported into another jupyter notebook.
X_test.to_csv (r'test_clean.csv', index = False)



---

### 2. Extract Latitude and Longitude


**Note:** Extracting the latitude and longitude did not work well for me. When I extracted Latitude and Longiitude, the accuracy score on the train dataset was 17% (yikes). One thing that could have contributed to a low score is that I only extracted lat/long for country and not to province. The reason for this is because there are a lot of provinces in this dataset that the geopy package did not recognize. If I spend more time on this project, I could try to remove the uncommon/unknown provinces and countries from the dataset. The benefits of adding lat and long is that they reference location with only two columns of data. Whereas the text dataset of country/province really enlarges the dataset as we have to use one hot encoding to transform the text dataset to numerical.

I installed a package called geopy which can be used to extract the latitude and longitude of a given city, country.

Link for reference:
https://pypi.org/project/geopy/

In [29]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=3)

/Users/avielstern/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [30]:
province = X_train.groupby(['province', 'country']).count().reset_index()['province']
country = X_train.groupby(['province', 'country']).count().reset_index()['country']

In [31]:
country = X_train.groupby('country').count().reset_index()['country']

In [32]:
# Define Latitude and Longitude varaibles which will be appended in the for loop
lat = []
long = []
location = {}

# Create a for loop to retrieve all lat/long that corresponds to each location of the wine
for i in np.arange(0, len(country)):
    
    # Locate latitude and longitude from region_1 and province
    location = geolocator.geocode(country[i])
    
    # append latitude and longitude
    lat.append(location.latitude)
    long.append(location.longitude)



In [33]:
latitude = {}
for i in np.arange(0, len(country)):
    latitude.update({country[i]:lat[i]})

longitude = {}
for i in np.arange(0, len(country)):
    longitude.update({country[i]:long[i]})

In [34]:
# Map lat and long as a dictionary to dataframe
X_train['latitude'] = X_train['country'].map(latitude)
# Map lat and long as a dictionary to dataframe
X_train['longitude'] = X_train['country'].map(longitude)

In [35]:
X_train.head()

,country,description,price,province,variety,vintage,points,latitude,longitude
0,France,An impressively structured blend of Tannat and...,25.0,Southwest France,Tannat-Cabernet Franc,2008.0,90,46.603354,1.888334
1,Argentina,"Roasted, earthy aromas of herbs and tomato pla...",14.0,Other,Cabernet Sauvignon,2013.0,85,-34.996496,-64.967282
2,Portugal,The southern coastal region of Algarve is bett...,35.0,Algarve,Portuguese Red,2014.0,90,40.033263,-7.889626
3,Australia,Aromas of sour cherry and cooked strawberries ...,30.0,Victoria,Pinot Noir,2006.0,82,-24.776109,134.755000
4,Italy,The Calderara Sottana cru delivers wines with ...,46.0,Sicily & Sardinia,Red Blend,2007.0,93,42.638426,12.674297


In [36]:
train = X_train.copy()

In [37]:
y_train = y_train.reset_index()
y_train.drop('index', axis = 1, inplace = True)

In [38]:
y_train.head()

,points
0,90
1,85
2,90
3,82
4,93


In [39]:
train['points'] = y_train['points']

In [40]:
train.head()

,country,description,price,province,variety,vintage,points,latitude,longitude
0,France,An impressively structured blend of Tannat and...,25.0,Southwest France,Tannat-Cabernet Franc,2008.0,90,46.603354,1.888334
1,Argentina,"Roasted, earthy aromas of herbs and tomato pla...",14.0,Other,Cabernet Sauvignon,2013.0,85,-34.996496,-64.967282
2,Portugal,The southern coastal region of Algarve is bett...,35.0,Algarve,Portuguese Red,2014.0,90,40.033263,-7.889626
3,Australia,Aromas of sour cherry and cooked strawberries ...,30.0,Victoria,Pinot Noir,2006.0,82,-24.776109,134.755000
4,Italy,The Calderara Sottana cru delivers wines with ...,46.0,Sicily & Sardinia,Red Blend,2007.0,93,42.638426,12.674297


### 3. BONUS - Extract average climate from that year and location from the wine

I will not complete this for my capstone, but I am placing this here to show that this is something to consider. As well as something I can try out later. The below dataset is taken from kaggle. This might be interesting to use if you want to try to predict withe wine score without the wine review. The idea would be to essentially remove the reviewer all together and be still be able to predict the score with contributing factors as location, year, climate, wine variety. 

In [832]:
temp_country = pd.read_csv('climate/GlobalLandTemperaturesByCountry.csv')
temp_major_city = pd.read_csv('climate/GlobalLandTemperaturesByMajorCity.csv')
temp_city = pd.read_csv('climate/GlobalLandTemperaturesByCity.csv')
temp_state = pd.read_csv('climate/GlobalLandTemperaturesByState.csv')
temp = pd.read_csv('climate/GlobalTemperatures.csv')